In [1]:
# Libraries
import googlemaps
import os
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, Literal, Optional
from tavily import TavilyClient
from pydantic import BaseModel, Field
from serpapi import GoogleSearch

In [ ]:
# API and environments
os.environ["GPLACES_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = "-"
os.environ["TAVILY_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
tavily_api = os.environ["TAVILY_API_KEY"]
api =os.environ["GPLACES_API_KEY"]
gmaps = googlemaps.Client(api)
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)
tavily_client = TavilyClient(tavily_api)

In [ ]:
def flight_info_extract(flight_option, departure_id: str, arrival_id: str, travel_date: str):
    all_options= []
    # other_flights = results['other_flights']
    if not flight_option:
        result = f"There are no Flights from: {departure_id} to {arrival_id} on date: {travel_date}"
        return all_options, result
    for idx, flight in enumerate(flight_option):
        price = flight.get("price", -1)
        if price >= 0:
            all_options.append(
                {
                    "Flight Option": idx+1,
                    "price": price,
                    "Layover": [layover_name.get('name', "") for layover_name in flight.get('layovers', [])] if flight.get('layovers') else ["No Layover"],
                    "Total Flight Duration (hours)": round(flight.get('total_duration', 0)/60, 1)
                }
            )
        else:
            continue
    all_options.sort(key = lambda x: x['price'])
    if not all_options:
        result = f"There is no Price available for all flights."
        return all_options, result 
    result = f"There are Flights from: {departure_id} to {arrival_id} on date: {travel_date}" 
    return all_options, result 


@tool
def One_way_flight_search(departure_id: str, arrival_id: str, travel_date: str):
    """
   Search for one-way flights between two airports on a specific date.
    Returns flight options with price, layovers, and duration details.
    
    Args:
        departure_id (str): Departure airport code (e.g., 'PEK' for Beijing)
        arrival_id (str): Arrival airport code (e.g., 'ISB' for Islamabad)
        travel_date (str): Travel date in YYYY-MM-DD format (e.g., '2026-03-03')
    
    Returns:
        str: Formatted flight search results with prices, durations, and layover information
    
    """
    params = {
    "engine": "google_flights",
    "departure_id": departure_id,
    "arrival_id": arrival_id,
    "currency": "USD",
    "type": "2",
    "outbound_date": travel_date,
    "api_key": ""
    }
    try:
        search = GoogleSearch(params)
        results = search.get_dict()
    except Exception as e:
        return f"Failed to Fetch Flight Details: {e}"

    if results.get('error'):
        return f"There is some issue with the results: {results['error']}"
    lowest_available_price = results['price_insights']['lowest_price']
    current_price_status = results['price_insights']['price_level']
    average_price_status= results['price_insights']['typical_price_range']
    departure_airport = results['airports'][0]['departure'][0]['airport']['name']
    arrival_airport = results['airports'][0]['arrival'][0]['airport']['name']
    best_flight_result, best_flight_remarks = flight_info_extract(results.get("best_flights", []), departure_airport, arrival_airport, travel_date)
    other_flight_result, other_flight_remarks = flight_info_extract(results.get("other_flights", []), departure_airport, arrival_airport, travel_date) 
    final_result = f"""
    Departure Airport {departure_airport}({departure_id})
    Arrival Airport {arrival_airport}({arrival_id})
    Lowest Available Price: {lowest_available_price}
    Current Prices Status: {current_price_status}
    Average Price Status: {average_price_status}
    Best Flight Information:
        There are total of {len(best_flight_result)} flight options.
        Remarks on Best Flight: {best_flight_remarks}
        Overall Best Flight Info: {best_flight_result}
    Other Flight Information:
        There are total of {len(other_flight_result)} flight options.
        Remarks on Other Flights: {other_flight_remarks}
        Overall Other Flight Info: {other_flight_result}
    """
    print(f"Flight search Tool: {final_result}")
    return final_result


In [5]:
flight_agent_prompt = """You are the FlightLogisticsSpecialist. Your sole purpose is to find one-way flight options using the One_way_flight_search tool.

TOOLS:
- One_way_flight_search(departure_id, arrival_id, travel_date): Searches for flights.

CRITICAL INPUT RULES:
1. Airport Codes: The tool REQUIRES IATA airport codes (e.g., "LHE" for Lahore, "PEK" for Beijing, "JFK" for New York).
   - If the user provides a city name, YOU MUST convert it to the correct IATA code before calling the tool.
2. Date Format: The tool REQUIRES dates in "YYYY-MM-DD" format.
   - If the user says "next Friday" or "March 3rd", convert it to "YYYY-MM-DD" relative to the current date.

OUTPUT GUIDELINES:
- Start with the "Lowest Available Price" and general price status.
- Present "Best Flight Options" first (these offer the best balance of price/duration).
- Summarize "Other Options" if they offer significantly cheaper prices but longer durations.
- Highlight layovers explicitly.

ERROR HANDLING:
- If the tool returns "Failed to Fetch" or "No Flights", apologize and ask the user to verify the date or route.

Example Call:
User: "Find flights from Lahore to Beijing on 5th Nov 2025"
Action: One_way_flight_search(departure_id="LHE", arrival_id="PEK", travel_date="2025-11-05")
"""

# Correcting your create_agent call:
# Note: You listed 'flight_info_extract' in tools, but that is a helper function. 
# The actual tool is 'One_way_flight_search'.
flight_agent = create_agent(
    model=model, 
    tools=[One_way_flight_search], 
    system_prompt=flight_agent_prompt
)

In [6]:
flight_agent.invoke({"messages": "Tell me flight details for beijing to shanghai on 2nd march 2026"})

Flight search Tool: 
    Departure Airport Beijing Capital International Airport(PEK)
    Arrival Airport Shanghai Hongqiao International Airport(SHA)
    Lowest Available Price: 223
    Current Prices Status: typical
    Average Price Status: [205, 230]
    Best Flight Information:
        There are total of 5 flight options.
        Remarks on Best Flight: There are Flights from: Beijing Capital International Airport to Shanghai Hongqiao International Airport on date: 2026-03-02
        Overall Best Flight Info: [{'Flight Option': 1, 'price': 223, 'Layover': ['Macau International Airport'], 'Total Flight Duration (hours)': 9.2}, {'Flight Option': 2, 'price': 349, 'Layover': ['No Layover'], 'Total Flight Duration (hours)': 2.2}, {'Flight Option': 3, 'price': 349, 'Layover': ['No Layover'], 'Total Flight Duration (hours)': 2.2}, {'Flight Option': 4, 'price': 349, 'Layover': ['No Layover'], 'Total Flight Duration (hours)': 2.1}, {'Flight Option': 5, 'price': 349, 'Layover': ['No Layover

{'messages': [HumanMessage(content='Tell me flight details for beijing to shanghai on 2nd march 2026', additional_kwargs={}, response_metadata={}, id='5ed36e72-5a51-421f-8358-c48faf64adf3'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'One_way_flight_search', 'arguments': '{"travel_date": "2026-03-02", "departure_id": "PEK", "arrival_id": "SHA"}'}, '__gemini_function_call_thought_signatures__': {'a1b93174-5903-47cc-a0cd-f2bf2984d856': 'CqwCAXLI2nxibAXtzYczAq32AIDE6D0OjfPEEApC/A/1rXFLh3im96o/jOejBF9lJtBkiU8/mjC49NOQzxigsIaizX3NLhQLokonsvDI7y6mgBFqKQLwZ04KfZJIBnk052tNqsn3C/y47EXVzNuFl7mBQUwNuZOFWNSDA/IIICCOZWfrkmGXKHBGNkHaZN/hoHIMf3DKdZYyPjKNikTCeTw1r8HKeRa1gdU4LEfV4z8LFU12fo0eeQKQeEGjAn/ZRg+cm6uKPKjdwmDzgt18aEANZUEGgpewDnFL1EufoZf7BFYIguXEuqchY4xLBMkD8Qci96n1qc/gkgtSWshcIKGZH7TyMB9fRcqhzJ7zfmOHWt+B+564eFxH2SOWJJZSCmbcORxYbHrowh18WFP1'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_